<a href="https://colab.research.google.com/github/silviolima07/app_b3/blob/main/app_b3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 120.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [2]:
!pip install pyngrok --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!pip install investpy --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 89.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 29.0 MB/s eta 0:00:00


In [4]:
#from pyngrok import ngrok

In [24]:
%%writefile app.py

import streamlit as st
from PIL import Image
import pandas as pd
import investpy as inv
import yfinance as yfin
yfin.pdr_override()
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import date
from prophet import Prophet
from bokeh.models.widgets import Div
import time

def get_ticker():
  br = inv.stocks.get_stocks(country='brazil')
  lista_tickers = []
  sufixo = '.SA'
  for i in [ticker for ticker in br.symbol]:
    lista_tickers.append(i+sufixo)
  return lista_tickers

def predict(ticker):
  yf = yfin.Ticker(ticker)
  symbol =  yf.info['symbol']
  description = yf.info['longName']
  #print("Stock: ", yf.info['symbol'])
  #print("Name: " , yf.info['longName'])
  hist = yf.history(period="max")
  hist = hist[['Close']]
  hist.reset_index(inplace=True)
  hist = hist.rename({'Date': 'ds', 'Close': 'y'}, axis='columns')
  hist['ds'] = hist['ds'].dt.tz_localize(None)
  m = Prophet(daily_seasonality=True)
  m.fit(hist)
  future = m.make_future_dataframe(periods=365)
  forecast = m.predict(future)
  return (symbol, description, forecast,m)  

def save_plot(symbol, description,forecast,m):
  print(symbol)
  print("Predictions\n\ty(future value)\tTrend\tWeakly\tYearly\tDaily")
  fig1 = m.plot(forecast)
  fig1.savefig('prophetplot1.png')
  st.markdown('### '+symbol+" -> "+description)
  st.markdown("### Prediction values")
  st.image('/content/prophetplot1.png')
  #
  st.markdown("### Components: Trends Weekly Yearly Daily")
  fig2 =  m.plot_components(forecast)
  fig2.savefig("prophetplot2.png")
  st.image('/content/prophetplot2.png')

def main():
    
    """B3 App """
    
    html_page = """
    <div style="background-color:tomato;padding=50px">
        <p style='text-align:center;font-size:50px;font-weight:bold'>Stocks & Prophet</p>
    </div>
              """
    st.markdown(html_page, unsafe_allow_html=True)

    image = Image.open("/content/drive/MyDrive/Logo_B3 .png")
    st.sidebar.image(image,caption="", use_column_width=True)
   
    activities = ["Predictions","About"]
    choice = st.sidebar.radio("Home",activities)
    Flag=False

    if choice == 'Predictions':
        #st.write("Files:")
        #st.markdown("### Files -> .txt")
        #st.write("After uploading you can convert to 7 languages")
        st.markdown("### Choose a ticker")
        option = st.selectbox('',get_ticker())
        #st.write('You selected:', option)
        if st.button("Predict next 365 days"):
           try:
             with st.spinner('Wait for it...'):
               symbol, description,forecast,model = predict(option)
               save_plot(symbol, description,forecast,model)
           except:
             st.error('Ops...sorry this ticker likely was renamed.', icon="🚨")
          
        
   

    if choice == 'About':
        st.subheader("I hope you enjoy it and use to learn something")
        st.markdown("### Referências")
        st.markdown("#### - https://analyzingalpha.com/yfinance-python")
        st.markdown("#### - https://analisemacro.com.br/mercado-financeiro/datareader-e-analises-com-yahoo-finance/")
        st.markdown("#### - https://levelup.gitconnected.com/how-to-plot-stock-prices-using-python-87ba684d126c")
        st.markdown("#### - https://www.youtube.com/watch?v=ZxLJjUcP2LI")
        st.markdown("#### - https://www.section.io/engineering-education/how-to-plot-a-candlestick-chart-in-python-using-the-matplotlib-finance-api/")
        st.markdown("#### - https://hareeshpb.medium.com/stock-prediction-using-prophet-python-525710e1ab0c")
        st.subheader("by Silvio Lima")
        
        if st.button("Linkedin"):
            js = "window.open('https://www.linkedin.com/in/silviocesarlima/')"
            html = '<img src onerror="{}">'.format(js)
            div = Div(text=html)
            st.bokeh_chart(div)



if __name__ == '__main__':
    main()

Overwriting app.py


In [8]:
#!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [9]:
#!streamlit run /content/app.py &>/content/logs.txt &

In [16]:
#ngrok.kill()

In [17]:
# Terminate ngrok port
#ngrok.kill()
# Set authentication (optional)
# Get your authentication token via https://dashboard.ngrok.com/auth
#
#NGROK_AUTH_TOKEN = "1gNjeFx7GPLDxTs5H60p5ZeZgl8_2NQZEtskwajdxd3ge8ZSx"
#ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [18]:
# Open an HTTPs tunnel on port 5000 for http://localhost:5000
#ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
#print("Streamlit Tracking UI:", ngrok_tunnel.public_url)

Streamlit Tracking UI: https://ffd3-34-150-223-252.ngrok-free.app
